In [343]:
import pandas as pd
import numpy as np
import json

# Importing test data now but will be from S3 buckets later

# Get rateTable from S3
rate_file = open('rateTable.json', "r") # will be from S3 not local
rateTableJson_object = json.load(rate_file)
rate_file.close()
#print(rateTableJson_object)

# Get correlMatrix from S3
correlMat_file = open('correlMat.json', "r") # will be from S3 not local
correlMatJson_object = json.load(correlMat_file)
correlMat_file.close()
#print(correlMatJson_object)

# Get protocolTierMap from S3
protocolTierMap_file = open('protocolTierMap.json', "r") # will be from S3 not local
protocolTierMap__object = json.load(protocolTierMap_file)
protocolTierMap_file.close()
#print(protocolTierMap__object)

# Get Account Postions from 3rd party API
# return a list of account positions in df format wtih balane, currency et al
accountPositionBalances_file = open('accountPositionBalances.json', "r") # will be from S3 not local
accountPositionBalances_object = json.load(accountPositionBalances_file)
accountPositionBalances_file.close()

# Creating dataframes
rateTable = pd.DataFrame(rateTableJson_object)
correlMat = pd.DataFrame(correlMatJson_object)
protocolTierMap = pd.DataFrame(protocolTierMap__object)

# TODO: Remove this function once 3rd party API is available and returning df
positions = accountPositionBalances_object['balances']['0x1fc6e73075c584dbdda0e53449e2c944986b9a72']['products']
# Init a dataframe to store an account's positions
portfolio = pd.DataFrame()
data = []

# Loop through each product and add to dataframe
for a in positions:
    assets = a['assets']
    for b in assets:
        tokens = b['tokens']
        for c in tokens:
            values = c['address'],c['symbol'],c['balance'],c['balanceUSD']
            data.append(values)
portfolio = portfolio.append(data,  ignore_index=True)
portfolio.columns =['address', 'symbol', 'balance', 'balanceUSD']

""" def getProtocolScore(portfolioInput):
    print("Hello from a getProtocolScore")  
    return portfolioInput
 """
def getPortocolTier(portfolioInput, protocolTierMapInput, rateTableInput):
    # Left join protocols in aacount with known protocol tiers then remove nans
    scored_portfolio = pd.merge(portfolioInput,protocolTierMapInput, left_on='address', right_on='address',how='left')
    #print(scored_portfolio) # TODO save this off to S3 but also need account number
    scored_portfolio['tier'] = scored_portfolio['tier'].replace(np.nan, 'tier4')  # sets unknown protocol to highest risk tier 
    # aggregate portfolio positions by tier
    balanceByTierOutput = scored_portfolio.groupby(['tier'])['balanceUSD'].sum().reset_index(name ='USD Balance')
    # Out join Rate Table and Balance By Tier then remove nans
    combinedTable = pd.merge(rateTableInput,balanceByTierOutput, left_on='tier', right_on='tier',how='left')
    combinedTable['USD Balance'] = combinedTable['USD Balance'].replace(np.nan, 0)   
    rp_column = combinedTable['USD Balance'] * combinedTable["rrol"]
    combinedTable['rp-usd'] = rp_column
    ra_column = combinedTable['rp-usd'] * combinedTable['riskLoad']
    combinedTable['risk-adj'] = ra_column
    return combinedTable

def calcPortfolioScore(portfolioInput, correlMatInput):
    # Calculate portfolio score
    portfolioBalanceBaseCurrency = portfolioInput['USD Balance'].sum()
    portfolioRPinBaseCurrency = portfolioInput['rp-usd'].sum()
    print("Portfolio Balance in Base Currency: " + str(portfolioBalanceBaseCurrency))
    print("Portfolio RP in Base Currency: " + str(portfolioRPinBaseCurrency))
    print(correlMatInput)
    # ########calculate portfolio rate using matmul and return it
    return #portfolioRateScore

balanceByTier = getPortocolTier(portfolio, protocolTierMap, rateTable)
print(balanceByTier)

print(calcPortfolioScore(balanceByTier,correlMat))
#print(getProtocolScore(portfolio))



    tier   rol  rrol  riskLoad  USD Balance      rp-usd   risk-adj
0  tier1  0.05  0.05       0.1  1265.937457   63.296873   6.329687
1  tier2  0.10  0.10       0.2   493.973324   49.397332   9.879466
2  tier3  0.15  0.15       0.3     0.000000    0.000000   0.000000
3  tier4  0.20  0.20       0.4   564.968728  112.993746  45.197498
Portfolio Balance in Base Currency: 2324.879509418006
Portfolio RP in Base Currency: 225.6879509418006
   names  tier1  tier2  tier3  tier4
0  tier1   1.00   0.10   0.15   0.20
1  tier2   0.10   1.00   0.20   0.15
2  tier3   0.15   0.20   1.00   0.40
3  tier4   0.20   0.15   0.40   1.00
None
